In [ ]:
# default_exp utils

# Utils
> Utilities

In [ ]:
# exporti
from functools import wraps

import jax
import jax.numpy as jnp
import numpy as np
from jax.lax import conv

## Convolutions

In [ ]:
# export
@wraps(conv)
def conv2d(lhs, rhs, window_strides=(1,1), padding="SAME", **kwargs):
    return conv(lhs[None, None, :, :], rhs[None, None, :, :], window_strides, padding, **kwargs)[0, 0, :, :]

In [ ]:
# export
@wraps(conv)
def batch_conv2d(lhs, rhs, window_strides=(1,1), padding="SAME", **kwargs):
    return conv(lhs[:, None, :, :], rhs[:, None, :, :], window_strides, padding, **kwargs)[:, 0, :, :]

In [ ]:
# export
def dilute(touches, brush):
    return conv2d(
        lhs=touches,
        rhs=brush,
        window_strides=(1, 1),
        padding="SAME",
        preferred_element_type=bool,
    )

## Random

I just can't be bothered doing this the JAX way...

In [ ]:
# export
def randn(shape, r=None, dtype=float):
    if r is not None:
        if isinstance(r, int):
            r = np.random.RandomState(seed=r)
    else:
        r = np.random
    return jnp.asarray(r.randn(*shape), dtype=dtype)

In [ ]:
# export
def rand(shape, r=None, dtype=float):
    if r is not None:
        if isinstance(r, int):
            r = np.random.RandomState(seed=r)
    else:
        r = np.random
    return jnp.asarray(r.rand(*shape), dtype=dtype)

## Argmax / Argmin

In [ ]:
# export
@jax.jit
def argmax2d(arr2d):
    m, n = arr2d.shape
    arr1d = arr2d.ravel()
    k = jnp.argmax(arr1d)
    return k//m, k%m

In [ ]:
# export
@jax.jit
def argmin2d(arr2d):
    m, n = arr2d.shape
    arr1d = arr2d.ravel()
    k = jnp.argmin(arr1d)
    return k//m, k%m